In [1]:
#!/usr/bin/python2

## UR5/UR10 Inverse Kinematics - Ryan Keating Johns Hopkins University


# ***** lib
import numpy as np
from numpy import linalg


import cmath
import math
from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi

global mat
mat=np.matrix


# ****** Coefficients ******


global d1, d2,d3,d4,d5,d6,a1,a2,a3,a4,a5,a6
d1 = 0.089159;
d2 = 0;
d3 = 0;
d4 = 0.10915;
d5 = 0.09465;
d6 = 0.0823;
a1 = 0;
a2 = -0.425;
a3 = -0.39225;
a4 = 0;
a5 = 0;
a6 = 0;

global d, a, alph

d = mat([0.089159, 0, 0, 0.10915, 0.09465, 0.0823]) #ur5mm
a =mat([0 ,-0.425 ,-0.39225 ,0 ,0 ,0]) #ur5 mm
alph = mat([math.pi/2, 0, 0, math.pi/2, -math.pi/2, 0 ])  #ur5 


# ************************************************** FORWARD KINEMATICS

def AH( n,th,c  ):

  T_a = mat(np.identity(4), copy=False)
  T_a[0,3] = a[0,n-1]
  T_d = mat(np.identity(4), copy=False)
  T_d[2,3] = d[0,n-1]

  Rzt = mat([[cos(th[n-1,c]), -sin(th[n-1,c]), 0 ,0],
	         [sin(th[n-1,c]),  cos(th[n-1,c]), 0, 0],
	         [0,               0,              1, 0],
	         [0,               0,              0, 1]],copy=False)
      

  Rxa = mat([[1, 0,                 0,                  0],
			 [0, cos(alph[0,n-1]), -sin(alph[0,n-1]),   0],
			 [0, sin(alph[0,n-1]),  cos(alph[0,n-1]),   0],
			 [0, 0,                 0,                  1]],copy=False)

  A_i = T_d * Rzt * T_a * Rxa
	    

  return A_i

def HTrans(th,c ):  
  A_1=AH( 1,th,c  )
  A_2=AH( 2,th,c  )
  A_3=AH( 3,th,c  )
  A_4=AH( 4,th,c  )
  A_5=AH( 5,th,c  )
  A_6=AH( 6,th,c  )
      
  T_06=A_1*A_2*A_3*A_4*A_5*A_6

  return T_06

# ************************************************** INVERSE KINEMATICS 

def invKine(desired_pos):# T60
  th = mat(np.zeros((6, 8)))
  P_05 = (desired_pos * mat([0,0, -d6, 1]).T-mat([0,0,0,1 ]).T)
  
  # **** theta1 ****
  
  psi = atan2(P_05[2-1,0], P_05[1-1,0])
  phi = acos(d4 /sqrt(P_05[2-1,0]*P_05[2-1,0] + P_05[1-1,0]*P_05[1-1,0]))
  #The two solutions for theta1 correspond to the shoulder
  #being either left or right
  th[0, 0:4] = pi/2 + psi + phi
  th[0, 4:8] = pi/2 + psi - phi
  th = th.real
  
  # **** theta5 ****
  
  cl = [0, 4]# wrist up or down
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH(1,th,c))
	      T_16 = T_10 * desired_pos
	      th[4, c:c+2] = + acos((T_16[2,3]-d4)/d6);
	      th[4, c+2:c+4] = - acos((T_16[2,3]-d4)/d6);

  th = th.real
  
  # **** theta6 ****
  # theta6 is not well-defined when sin(theta5) = 0 or when T16(1,3), T16(2,3) = 0.

  cl = [0, 2, 4, 6]
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH(1,th,c))
	      T_16 = linalg.inv( T_10 * desired_pos )
	      th[5, c:c+2] = atan2((-T_16[1,2]/sin(th[4, c])),(T_16[0,2]/sin(th[4, c])))
		  
  th = th.real

  # **** theta3 ****
  cl = [0, 2, 4, 6]
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH(1,th,c))
	      T_65 = AH( 6,th,c)
	      T_54 = AH( 5,th,c)
	      T_14 = ( T_10 * desired_pos) * linalg.inv(T_54 * T_65)
	      P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0,0,0,1]).T
	      t3 = cmath.acos((linalg.norm(P_13)**2 - a2**2 - a3**2 )/(2 * a2 * a3)) # norm ?
	      th[2, c] = t3.real
	      th[2, c+1] = -t3.real

  # **** theta2 and theta 4 ****

  cl = [0, 1, 2, 3, 4, 5, 6, 7]
  for i in range(0,len(cl)):
	      c = cl[i]
	      T_10 = linalg.inv(AH( 1,th,c ))
	      T_65 = linalg.inv(AH( 6,th,c))
	      T_54 = linalg.inv(AH( 5,th,c))
	      T_14 = (T_10 * desired_pos) * T_65 * T_54
	      P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0,0,0,1]).T
	      
	      # theta 2
	      th[1, c] = -atan2(P_13[1], -P_13[0]) + asin(a3* sin(th[2,c])/linalg.norm(P_13))
	      # theta 4
	      T_32 = linalg.inv(AH( 3,th,c))
	      T_21 = linalg.inv(AH( 2,th,c))
	      T_34 = T_32 * T_21 * T_14
	      th[3, c] = atan2(T_34[1,0], T_34[0,0])
  th = th.real

  return th

def AHD( n,th ):

  T_a = mat(np.identity(4), copy=False)
  T_a[0,3] = a[0,n-1]
  T_d = mat(np.identity(4), copy=False)
  T_d[2,3] = d[0,n-1]

  Rzt = mat([[cos(th[n-1]), -sin(th[n-1]), 0 ,0],
	         [sin(th[n-1]),  cos(th[n-1]), 0, 0],
	         [0,               0,              1, 0],
	         [0,               0,              0, 1]],copy=False)
      

  Rxa = mat([[1, 0,                 0,                  0],
			 [0, cos(alph[0,n-1]), -sin(alph[0,n-1]),   0],
			 [0, sin(alph[0,n-1]),  cos(alph[0,n-1]),   0],
			 [0, 0,                 0,                  1]],copy=False)

  A_i = T_d * Rzt * T_a * Rxa
	    

  return A_i

def fwkinematics(th ):  
  A_1=AHD( 1,th)
  A_2=AHD( 2,th)
  A_3=AHD( 3,th)
  A_4=AHD( 4,th)
  A_5=AHD( 5,th )
  A_6=AHD( 6,th)
      
  T_06=A_1*A_2*A_3*A_4*A_5*A_6

  return T_06

In [2]:
th= mat(np.zeros((6, 8)))
th[0,0]=0.2
th[1,0]=0.2
th[2,0]=0.2
th[3,0]=0.2
th[4,0]=0.2
th[5,0]=0.2
print(th)

[[0.2 0.  0.  0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.  0.  0.  0. ]]


In [3]:
desired=HTrans(th,0 )
desired

matrix([[ 0.70569928, -0.7076948 ,  0.03400876, -0.68544819],
        [-0.055617  , -0.10318457, -0.99310608, -0.33261721],
        [ 0.70632519,  0.69894278, -0.11217714, -0.23537501],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [4]:
thc=invKine(desired)
print(thc[3,0])
print(thc)

0.19999999999999896
[[ 0.2         0.2         0.2         0.2        -2.64247804 -2.64247804
  -2.64247804 -2.64247804]
 [ 0.2         0.39195854  0.23092179  0.82977026  2.49538033  2.73587689
   2.56096765  3.14109144]
 [ 0.2        -0.2         0.62472939 -0.62472939  0.25059141 -0.25059141
   0.6051407  -0.6051407 ]
 [ 0.2         0.40804146  2.88594148 -2.74663352  0.14894232  0.40962858
   2.87039836 -2.78262933]
 [ 0.2         0.2        -0.2        -0.2         2.66427858  2.66427858
  -2.66427858 -2.66427858]
 [ 0.2         0.2        -2.94159265 -2.94159265 -2.57098266 -2.57098266
   0.57061     0.57061   ]]


In [5]:
for c in range(8):
    print('\n',HTrans(thc,c))


 [[ 0.70569928 -0.7076948   0.03400876 -0.68544819]
 [-0.055617   -0.10318457 -0.99310608 -0.33261721]
 [ 0.70632519  0.69894278 -0.11217714 -0.23537501]
 [ 0.          0.          0.          1.        ]]

 [[ 0.70569928 -0.7076948   0.03400876 -0.68544819]
 [-0.055617   -0.10318457 -0.99310608 -0.33261721]
 [ 0.70632519  0.69894278 -0.11217714 -0.23537501]
 [ 0.          0.          0.          1.        ]]

 [[ 0.70569928 -0.7076948   0.03400876 -0.68544819]
 [-0.055617   -0.10318457 -0.99310608 -0.33261721]
 [ 0.70632519  0.69894278 -0.11217714 -0.23537501]
 [ 0.          0.          0.          1.        ]]

 [[ 0.70569928 -0.7076948   0.03400876 -0.68544819]
 [-0.055617   -0.10318457 -0.99310608 -0.33261721]
 [ 0.70632519  0.69894278 -0.11217714 -0.23537501]
 [ 0.          0.          0.          1.        ]]

 [[ 0.70569928 -0.7076948   0.03400876 -0.68544819]
 [-0.055617   -0.10318457 -0.99310608 -0.33261721]
 [ 0.70632519  0.69894278 -0.11217714 -0.23537501]
 [ 0.          0.

In [6]:
import sim as vrep
import numpy as np
from numpy import linalg
import time
import math
import sys
from scipy.spatial.transform import Rotation as R
from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi
from time import sleep


In [7]:
rot_mat=[[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3]]
rot_mat[2][3]

2

In [8]:

def eulerzyxinv(rot_mat) :
    theta=[0,0,0]
    if abs(rot_mat[2][2])<1e-20 and abs(rot_mat[1][2])<1e-20:
       theta[0]=0
       theta[1]=math.atan2(rot_mat[0][2],rot_mat[2][2])
       theta[2]=math.atan2(rot_mat[1][0],rot_mat[1][1])
    else:
        theta[0]=math.atan2(-rot_mat[1][2],rot_mat[2][2])
        sinr=math.sin(theta[0])
        cosr=math.cos(theta[0])
        theta[1]=math.atan2(rot_mat[0][2], cosr * rot_mat[2][2] - sinr * rot_mat[1][2])
        theta[2]=math.atan2(-rot_mat[0][1],rot_mat[0][0])
 
    return theta

def copyf(clientID,g,h,herl):
    rot_mat=[[g[0][0],g[0][1],g[0][2]],[g[1][0],g[1][1],g[1][2]],[g[2][0],g[2][1],g[2][2]]]
    pos=[g[0][3],g[1][3],g[2][3]]
    theta= eulerzyxinv(rot_mat)
    errorCode, hnew = vrep.simxCopyPasteObjects(clientID, h, vrep.simx_opmode_oneshot_wait)
    errorCode, res = vrep.simxSetObjectOrientation(clientID, hnew, herl, theta, vrep.simx_opmode_oneshot)
    errorCode, res = vrep.simxSetObjectPosition(clientID, hnew, herl, pos, vrep.simx_opmode_oneshot)
    return hnew

def movef(clientID,g,h,herl):
    rot_mat=[[g[0][0],g[0][1],g[0][2]],[g[1][0],g[1][1],g[1][2]],[g[2][0],g[2][1],g[2][2]]]
    pos=[g[0][3],g[1][3],g[2][3]]
    theta= eulerzyxinv(rot_mat)
    errorCode, res = vrep.simxSetObjectOrientation(clientID, hnew, herl, theta, vrep.simx_opmode_oneshot)
    errorCode, res = vrep.simxSetObjectPosition(clientID, hnew, herl, pos, vrep.simx_opmode_oneshot)
    return

In [9]:
print ('UR5 MOVER started')
vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to V-REP
if clientID != -1:
    print ('Connected to UR5 Simulator')
    startingJoints = [np.pi/2, np.pi/2, 0, np.pi/2, 0, 0]
    Desired_T=[[1,0,0,-0.5],[0,0,-1,-0.4],[0,1,0,0.6],[0,0,0,1]]
    print("desired T",Desired_T)
    thc=invKine(Desired_T)
    # Joint:          1  2  3  4  5  6  
    #calculate the invKinematics theta list(a column(6*1 size) is a set of possible solution thetas)
    # List to store our joint handles in
    joint_handles = []

    # Loop through all the robot's joints and get unique ID's for each
    for i in range(6):
        joint_name = 'UR5_joint' + str(i + 1)
        errorCode, handle = vrep.simxGetObjectHandle(clientID, joint_name, vrep.simx_opmode_oneshot_wait)
        joint_handles.append(handle)
    errorCode, pos = vrep.simxGetObjectPosition(clientID,joint_handles[5],joint_handles[0],vrep.simx_opmode_oneshot_wait)
    print("Initial end pos:",pos)
    for i in range (6):
        vrep.simxSetJointTargetPosition(clientID,joint_handles[i],startingJoints[i], vrep.simx_opmode_oneshot_wait)
    
    for i in range(6):
        print("The calculated invKinematics theta",i,'is',thc[i,0])
        
    input("Press any key to move to target location")
    
    # Set the position of each joint
    for i in range (6):
        vrep.simxSetJointTargetPosition(clientID,joint_handles[i],thc[i,0], vrep.simx_opmode_oneshot_wait)
        # print("Setting joint", i+1, "to", theta_list[i])

    sleep(2)

    errorCode, pos_A = vrep.simxGetObjectPosition(clientID,joint_handles[5],joint_handles[0],vrep.simx_opmode_oneshot_wait)
    print("Actual pos:", pos_A)
   # errorCode, angles = vrep.simxGetObjectQuaternion(clientID,joint_handles[5],joint_handles[0],vrep.simx_opmode_oneshot_wait)
   # print("Actual pose:",pos_A)

    for i in range(6):
        theta = vrep.simxGetJointPosition(clientID, joint_handles[i], vrep.simx_opmode_oneshot_wait)
        print("Actual Theta",i,"is",theta)

    # sleep(10)
    input("Press any key to revert to origin...")
    print("")

    for i in range (6):
        vrep.simxSetJointTargetPosition(clientID,joint_handles[i],0,vrep.simx_opmode_oneshot_wait)
        print("Setting joint", i+1, "back to 0")

    sleep(0.5)
    vrep.simxFinish(clientID)
else:
    print ('Failed connecting to remote API server')
    sys.exit("Connection Failed")
print ('Program ended')

'''
    # set the ur5 frame
    errorCode, ur5Ref = vrep.simxGetObjectHandle(clientID, 'UR5',vrep.simx_opmode_oneshot_wait)
    #Construct the Base frame
    base = vrep.simxGetObjectHandle(clientID,'Frame0', vrep.simx_opmode_oneshot_wait)
    
    errorCode,FrameEnd=copyf(clientID,np.identity(4),base,base)
    errorCode,FrameEndTarget=copyf(clientID,np.identity(4),base,base)
    #other initialization stuff
    for i in range(6):
        errorCode,res_pos = vrep.simxGetJointPosition(clientID,joint_handles[i],vrep.simx_opmode_streaming)

    errorCode, res_ur5_pos = vrep.simxGetObjectPosition(clientID,ur5Ref, -1,vrep.simx_opmode_streaming)
    errorCode, res_ur5_ori = vrep.simxGetObjectOrientation(clientID,ur5Ref, -1,vrep.simx_opmode_streaming)

        
    threshold = 0.01
    startingJoints = [np.pi/2, np.pi/2, 0, np.pi/2, 0, 0]
    targetJoints=[1, 1, 1, 1, 1, 1]
    diff=[0,0,0,0,0,0]
    for i in range(6):
        diff[i]=targetJoints[i]-startingJoints[i]
    #set the arm to the starting configuration
    for i in range (6):
        vrep.simxSetJointTargetPosition(clientID,joint_handles[i],startingJoints[i], vrep.simx_opmode_oneshot_wait)
    
    G=fwkinematics(diff)
    movef(clientID,G,FrameEndTarget,base)
    thc=invKine(G)
    for i in range(8):
        print('Approaching the NO.',i,'theta solution\n')
        targetJointsActual=[0,0,0,0,0,0]
        for k in range(6):
            targetJointsActual[k]=thc[k,i]+startingJoints[k]
        for j in range(6):
            if targetJointsActual[j]>np.pi:
                targetJointsActual[j]=targetJointsActual[j]-2*np.pi
            elif targetJointsActual[j] < -np.pi:
                targetJointsActual[j]=targetJointsAcutal[j]+2*np.pi
            #print(targetJointsAcutal)
        for tj in range(6):
            vrep.simxSetJointTargetPosition(clientID,joint_handles[tj],
            targetJointsActual[tj],
            vrep.simx_opmode_oneshot)
        currentJoints=[0,0,0,0,0,0]
        reached=false
        while reached==false:
            for i in range(6):
                errorCode,currentJoints[i]=vrep.simxGetJointPosition(clientID,joint_handles[i],vrep.simx_opmode_oneshot_wait)
            diff_G=[0,0,0,0,0,0]
            for i in range(6):
                diff_G[i]=currentJoints[i]-startingJoints[i]
            G_current=fwkinematics(diff_G)
            movef(clientID,G_current,FrameEnd,base)
            diffJoints=[0,0,0,0,0,0]
            for i in range(6):
                diffJoints[i]=currentJoints[i]-targetJointsActual[i]
            for i in range(6):
                if diffJoints[i]>np.pi:
                     diffJoints[i]=diffJoints[i]-2*np.pi
                elif diffJoints[i]<-np.pi:
                     diffJoints[i]=diffJoints[i]+2*np.pi
            if max(abs(diffJoints))<threshold:
                reached=true
        print('NO.',i,'theta solution reached\n')
        sleep(2)
        for i in range(6):
            errorCode,res=vrep.simxSetJointTargetPosition(clientID,joint_handles[i],
                startingJoints(i),
                vrep.simx_opmode_oneshot)
    
        currentJoints=[0,0,0,0,0,0]
        reached=false
        while reached==false:
            for i in range(6):
                errorCode,currentJoints[i]=vrep.simxGetJointPosition(clientID,joint_handles[i],vrep.simx_opmode_oneshot_wait)
            diff_G=[0,0,0,0,0,0]
            for i in range(6):
                diff_G[i]=currentJoints[i]-startingJoints[i]
            G_current=fwkinematics(diff_G)
            movef(clientID,G_current,FrameEnd,base)
            diffJoints=[0,0,0,0,0,0]
            for i in range(6):
                diffJoints[i]=currentJoints[i]-startingJoints[i]
            for i in range(6):
                if diffJoints[i]>np.pi:
                     diffJoints[i]=diffJoints[i]-2*np.pi
                elif diffJoints[i]<-np.pi:
                     diffJoints[i]=diffJoints[i]+2*np.pi
            if max(abs(diffJoints))<threshold:
                reached=true
        sleep(2)
    sleep(5)
    vrep.simxRemoveObject(clientID, FrameEnd, vrep.simx_opmode_oneshot)
    vrep.simxRemoveObject(clientID, FrameEndTarget, vrep.simx_opmode_oneshot)
    vrep.simxFinish(clientID)
else:
    print ('Failed connecting to remote API server')
    sys.exit("Connection Failed")
print ('Program ended')    
    
   
'''



UR5 MOVER started
Connected to UR5 Simulator
desired T [[1, 0, 0, -0.5], [0, 0, -1, -0.4], [0, 1, 0, 0.6], [0, 0, 0, 1]]
Initial end pos: [-0.8351166844367981, 0.04902160167694092, 0.12114202231168747]
The calculated invKinematics theta 0 is 0.3807325830990953
The calculated invKinematics theta 1 is -0.804959744339629
The calculated invKinematics theta 2 is 0.0
The calculated invKinematics theta 3 is 0.8049597443396291
The calculated invKinematics theta 4 is 0.38073258309908625
The calculated invKinematics theta 5 is -1.6477993979288982e-16
Press any key to move to target location
Actual pos: [0.050729066133499146, -0.7081749439239502, 0.6092700958251953]
Actual Theta 0 is (0, 0.38072657585144043)
Actual Theta 1 is (0, -0.8051505088806152)
Actual Theta 2 is (0, 4.76837158203125e-07)
Actual Theta 3 is (0, 0.8048036098480225)
Actual Theta 4 is (0, 0.3807368278503418)
Actual Theta 5 is (0, 0.0)
Press any key to revert to origin...

Setting joint 1 back to 0
Setting joint 2 back to 0
Setti

'\n    # set the ur5 frame\n    errorCode, ur5Ref = vrep.simxGetObjectHandle(clientID, \'UR5\',vrep.simx_opmode_oneshot_wait)\n    #Construct the Base frame\n    base = vrep.simxGetObjectHandle(clientID,\'Frame0\', vrep.simx_opmode_oneshot_wait)\n    \n    errorCode,FrameEnd=copyf(clientID,np.identity(4),base,base)\n    errorCode,FrameEndTarget=copyf(clientID,np.identity(4),base,base)\n    #other initialization stuff\n    for i in range(6):\n        errorCode,res_pos = vrep.simxGetJointPosition(clientID,joint_handles[i],vrep.simx_opmode_streaming)\n\n    errorCode, res_ur5_pos = vrep.simxGetObjectPosition(clientID,ur5Ref, -1,vrep.simx_opmode_streaming)\n    errorCode, res_ur5_ori = vrep.simxGetObjectOrientation(clientID,ur5Ref, -1,vrep.simx_opmode_streaming)\n\n        \n    threshold = 0.01\n    startingJoints = [np.pi/2, np.pi/2, 0, np.pi/2, 0, 0]\n    targetJoints=[1, 1, 1, 1, 1, 1]\n    diff=[0,0,0,0,0,0]\n    for i in range(6):\n        diff[i]=targetJoints[i]-startingJoints[i]\n 